# Interpolation

PODPAC supports the following interpolation methods with the associated Interpolator classes:
    
```python
INTERPOLATION_METHODS = {
    'optimal': [OptimalInterpolation],
    'nearest_preview': [NearestPreview],
    'nearest': [NearestNeighbor, Rasterio, ScipyGrid, ScipyPoint],
    'bilinear':[Rasterio, ScipyGrid],
    'cubic':[Rasterio],
    'cubic_spline':[Rasterio, ScipyGrid],
    'lanczos':[Rasterio],
    'average':[Rasterio],
    'mode':[Rasterio],
    'gauss':[Rasterio],
    'max':[Rasterio],
    'min':[Rasterio],
    'med':[Rasterio],
    'q1':[Rasterio],
    'q3': [Rasterio],
    'spline_2': [ScipyGrid],
    'spline_3': [ScipyGrid],
    'spline_4': [ScipyGrid],
    'radial': [Radial]
}
```

In [1]:
# notebook imports
from IPython.display import display
import numpy as np
from podpac.coordinates import Coordinates, clinspace
from podpac.data import Array

## Nearest Neighbor

In [39]:
# data source 
source = np.arange(0, 9)
source.resize((3, 3))
native_coords = Coordinates([[0, 5, 10], clinspace('2018-01-01', '2018-01-18', 3)], 
                             dims=['lat', 'time'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 9], clinspace('2018-01-01', '2018-01-09', 3)], 
                          dims=['lat', 'time'])

# show node source
node.source

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [40]:
# simple interpolation definition
node.interpolation = 'nearest'
output = node.eval(eval_coords)

# elements are chosen based on their nearest neighbor
output.data

array([[0., 1., 1.],
       [0., 1., 1.],
       [6., 7., 7.]])

In [41]:
# show the interpolation class for this node (generated by definition `node.interpolation = 'nearest'`)
node.interpolation_class

Interpolation
	('default',): nearest, ['NearestNeighbor', 'Rasterio', 'ScipyGrid', 'ScipyPoint'], {}

In [42]:
# show the interpolators that were used to do this interpolation
# this is only available *after* a node has been evaluated
node.interpolators

OrderedDict([(('lat', 'time'),
              <podpac.core.data.interpolate.NearestNeighbor at 0x1c1db595f8>)])

#### Spatial Tolerance

In [43]:
# set a space tolerance on interpolation
node.interpolation = {
    'method': 'nearest',
    'params': {
        'spatial_tolerance': 1.1
    }
}

output = node.eval(eval_coords)

# the 1st element is outside of the space tolerance
output.data

array([[ 0.,  1.,  1.],
       [nan, nan, nan],
       [ 6.,  7.,  7.]])

#### Time Tolerance

In [44]:
# set a time tolerance on interpolation
node.interpolation = {
        'method': 'nearest',
        'params': {
            'spatial_tolerance': 1.1,
            'time_tolerance': np.timedelta64(1, 'D')
        }
}

output = node.eval(eval_coords)
output.data

array([[ 0., nan,  1.],
       [nan, nan, nan],
       [ 6., nan,  7.]])

## Rasterio Interpolation

Rasterio provides interpolation support for more advanced interpolation methods.
The following interpolation methods are supported by rasterio via podpac:

```python
['nearest', 
 'bilinear', 
 'cubic', 
 'cubic_spline', 
 'lanczos', 
 'average', 
 'mode', 
 'gauss', 
 'max', 
 'min', 
 'med', 
 'q1', 
 'q3']
```

In [34]:
# data source
source = np.arange(0, 15)
source.resize((3, 5))

native_coords = Coordinates([clinspace(0, 10, 3), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 3), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [35]:
# max interpolation from rasterio
node.interpolation = 'max'
output = node.eval(eval_coords)
output.data

array([[ 6.,  7.,  8.,  9.,  9.],
       [11., 12., 13., 14., 14.],
       [11., 12., 13., 14., 14.]])

In [36]:
# min interpolation
node.interpolation = 'min'
output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.]])

In [37]:
# bilinear
node.interpolation = 'bilinear'
output = node.eval(eval_coords)
output.data

array([[ 1.4,  2.4,  3.4,  4.4,  5. ],
       [ 6.4,  7.4,  8.4,  9.4, 10. ],
       [10.4, 11.4, 12.4, 13.4, 14. ]])

In [38]:
# cubic_spline
node.interpolation = 'cubic_spline'
output = node.eval(eval_coords)
output.data

array([[ 2.01204892,  2.95976676,  3.95976676,  4.94251609,  5.49712175],
       [ 6.44026614,  7.38798398,  8.38798398,  9.3707333 ,  9.92533897],
       [ 9.85637899, 10.80409683, 11.80409683, 12.78684616, 13.34145182]])

## Specify Interpolator (Advanced)

You can specify a single type of interpolator for a data source using the `interpolators` key of the interpolation dictionary definition.

In [46]:
from podpac.interpolators import NearestNeighbor, Rasterio, ScipyGrid, ScipyPoint

In [47]:
# data source
source = np.arange(0, 25)
source.resize((5, 5))

native_coords = Coordinates([clinspace(0, 10, 5), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 5), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [51]:
# use Scipy for nearest neighbor
node.interpolation = {
    'method': 'nearest',
    'interpolators': [ScipyGrid]
}
output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  3., nan],
       [ 5.,  6.,  7.,  8., nan],
       [10., 11., 12., 13., nan],
       [15., 16., 17., 18., nan],
       [nan, nan, nan, nan, nan]])

In [52]:
# use Scipy for cubic_spline
node.interpolation = {
    'method': 'cubic_spline',
    'interpolators': [ScipyGrid]
}
output = node.eval(eval_coords)
output.data

array([[ 2.4,  3.4,  4.4,  5.4,  6. ],
       [ 7.4,  8.4,  9.4, 10.4, 11. ],
       [12.4, 13.4, 14.4, 15.4, 16. ],
       [17.4, 18.4, 19.4, 20.4, 21. ],
       [20.4, 21.4, 22.4, 23.4, 24. ]])

In [55]:
source = np.random.rand(5, 5, 3)
coords_src = Coordinates(
    [clinspace(0, 10, 5), clinspace(0, 10, 5), [2, 3, 5]], dims=['lat', 'lon', 'time'])
coords_dst = Coordinates(
    [clinspace(1, 11, 5), clinspace(1, 11, 5), [2, 3, 5]], dims=['lat', 'lon', 'time'])

node = Array(source=source, native_coordinates=coords_src, interpolation={
    'method': 'nearest',
    'interpolators': [ScipyGrid]
})
output = node.eval(coords_dst)
output

InterpolationException: Dimensions ['lon', 'lat', 'time'] can't be handled by interpolation definition:
 Interpolation
	('default',): nearest, ['ScipyGrid'], {}